In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from nltk.corpus import stopwords
import nltk

%matplotlib inline

C:\Users\mohit\Anaconda3\envs\py35\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#data = pd.read_json('D:/dataset/Yahoo QA Topic Classification.json', lines=True)
data =pd.read_csv("C:/Users/mohit/Desktop/NN/sentiment_data_5l.csv")
data.head()

,Unnamed: 0,sentiment,content
0,1631,hate,My stomach is killing me do j can't sleep
1,8481,anger,@anz_rocks19 i have to break the twitterparty...
2,763,hate,Taking back the HORRIBLE shoes my mum made me ...
3,396,sadness,my stupid tooooth hurts
4,18851,anger,too sick for rigging tomorrow.


In [3]:
stop_words = set(stopwords.words('english'))

In [4]:
word_embedding =open('D:/word_embedding/pre_trained_gloVe/glove.6B/glove.6B.50d.txt',encoding='utf8')

In [5]:
def read_glove_vecs(glovefile):
    word_list = []
    word_to_index = {}
    index_to_word = {}
    word_to_vec_map={}
    index = 0
    for line in glovefile: 
        word =line.split(" ")
        word_vec_list =np.float_(word[1:51])
        word_list.append(word[0])
        word_to_vec_map[word[0]] =word_vec_list
        word_to_index[word[0]] = index
        index = index + 1
        
    index_to_word = dict(zip(word_to_index.values(), word_to_index.keys()))
    
    return (word_to_index, index_to_word, word_to_vec_map, word_list)

def cosine_similarity(u,v):
    
    dot=np.dot(u,v)
    u_mod=np.sqrt(np.sum(np.multiply(u,u)))
    v_mod=np.sqrt(np.sum(np.multiply(v,v)))
    
    similarity =dot/(u_mod * v_mod)
    return similarity

def sentence_to_avg(sentence, word_to_vec_map):
    
    sentence_lower = sentence.lower()
    sentence_words = nltk.word_tokenize(sentence_lower)
    
    avg = np.zeros((50,))
    sent_len_counter = 1
    for word in sentence_words:
        if not word in stop_words:
            if word_to_vec_map.get(word) is not None:
                avg += word_to_vec_map.get(word)
                sent_len_counter = sent_len_counter + 1
        
    avg = avg/sent_len_counter
    
    return avg
        
def softmax(z):
    e_z = np.exp(z - np.max(z))
    return e_z / e_z.sum(axis=0)

def predict(X, Y_oh, W, b, word_to_vec_map, lebal_encoder_class):
    
    m = Y_oh.shape[0]
    predict = []
    for sentence_index in range(m):
        avg = sentence_to_avg(X[sentence_index], word_to_vec_map)   
        z = np.dot(W,avg)+b
        a = softmax(z)
        list_values = list(a)
        pred_index = list_values.index(max(list_values))
        pred_sentiment = lebal_encoder_class[pred_index]
        predict.append(pred_sentiment)
    return predict 

def predict1(X, W, b, word_to_vec_map, lebal_encoder_class):
    
    m = X.shape[0]
    predict = []
    for sentence_index in range(m):
        avg = sentence_to_avg(X[sentence_index], word_to_vec_map)   
        z = np.dot(W,avg)+b
        a = softmax(z)
        list_values = list(a)
        pred_index = list_values.index(max(list_values))
        pred_sentiment = lebal_encoder_class[pred_index]
        predict.append(pred_sentiment)
    return predict 

In [6]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=0)

In [7]:
print(len(train_data))
print(len(test_data))

396
99


In [8]:
le = LabelEncoder()
train_data_le_y = le.fit_transform(train_data[["sentiment"]])
test_data_le_y = le.transform(test_data[["sentiment"]])
le_classes = le.classes_
print(le_classes)

['anger' 'happiness' 'hate' 'love' 'sadness']


C:\Users\mohit\Anaconda3\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\mohit\Anaconda3\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
print(type(train_data_le_y))
print(train_data_le_y)

<class 'numpy.ndarray'>
[2 2 3 3 0 4 2 4 0 2 4 3 2 4 0 3 0 3 4 4 4 3 4 2 4 2 3 1 1 2 1 2 3 0 0 3 3
 3 1 0 3 0 1 0 1 0 0 0 1 3 3 3 0 2 4 0 2 3 0 4 0 1 0 1 2 2 4 4 0 0 1 4 1 3
 0 0 0 1 4 1 4 3 4 0 2 1 1 4 1 4 0 0 0 0 3 3 4 0 2 1 0 4 3 1 4 3 1 1 0 3 4
 1 1 3 1 2 3 4 3 1 1 3 1 0 0 3 4 2 4 0 4 1 0 1 0 0 1 1 3 4 0 4 4 2 0 0 2 1
 2 2 4 2 2 3 3 3 0 2 1 2 0 3 4 2 1 3 3 0 4 2 0 4 2 2 1 3 0 0 0 4 2 4 4 2 0
 3 2 4 2 1 3 2 3 2 4 1 0 0 1 0 1 3 0 1 3 0 1 4 2 0 1 0 4 2 3 2 1 0 3 3 4 0
 0 4 1 1 4 0 2 0 3 0 4 1 0 3 3 4 3 1 4 2 2 2 3 3 4 4 1 0 4 1 0 3 2 1 1 4 0
 1 4 4 2 2 1 2 2 3 4 3 3 2 4 3 3 0 1 3 1 3 3 4 0 4 2 3 3 1 3 1 0 4 4 4 2 3
 2 1 0 3 2 1 2 4 2 2 2 1 4 1 3 0 2 1 3 4 1 0 2 4 1 1 1 4 3 1 1 4 1 0 2 3 1
 0 4 4 1 4 1 3 4 3 0 0 1 2 1 2 2 2 2 0 4 2 0 4 4 1 4 2 1 1 2 4 0 4 4 3 1 2
 3 4 0 0 0 0 1 0 2 3 2 2 0 3 0 2 2 3 2 2 3 2 3 3 4 1]


In [10]:
word_to_index, index_to_word, word_to_vec_map, word_list = read_glove_vecs(word_embedding)

In [11]:
print(len(word_to_index))
print(len(index_to_word))
print(len(word_to_vec_map))
print(len(word_list))

400000
400000
400000
400000


In [12]:
word = "krishna"
index = 12782
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(index) + "th word in the vocabulary is", index_to_word[index])

the index of krishna in the vocabulary is 12782
the 12782th word in the vocabulary is krishna


In [13]:
def model(X, Y_oh, word_to_vec_map, lebal_encoder_class, learning_rate = 0.01, num_iterations = 400):
    
    np.random.seed(1)

    # Define number of training examples
    m = Y_oh.shape[0]                          # number of training examples
    n_y = Y_oh.shape[1]                                 # number of classes  
    n_h = 50                                # dimensions of the GloVe vectors 
    
    # Initialize parameters using Xavier initialization
    W = np.random.randn(n_y, n_h) / np.sqrt(n_h)
    b = np.zeros((n_y,))
    
    #Y_oh = convert_to_one_hot(Y, C = n_y)
    
    for t in range(num_iterations):
        
        for sentence_index in range(m):
            avg = sentence_to_avg(X[sentence_index], word_to_vec_map)
            
            z = np.dot(W,avg)+b
            a = softmax(z)
            
             
            cost = -np.sum(np.multiply(Y_oh[sentence_index], np.log(a)))
            
            
            dz = a - Y_oh[sentence_index]
            dW = np.dot(dz.reshape(n_y,1), avg.reshape(1, n_h))
            db = dz

            # Update parameters with Stochastic Gradient Descent
            W = W - learning_rate * dW
            b = b - learning_rate * db
            
        if t % 100 == 0:
            print("Epoch: " + str(t) + " --- cost = " + str(cost))
            pred = predict(X, Y_oh, W, b, word_to_vec_map, lebal_encoder_class)

    return pred, W, b

In [14]:
X_train = np.asarray(['I am going to the bar tonight', 'I love you', 'miss you my dear',
 'Lets go party and drinks','Congrats on the new job','Congratulations',
 'I am so happy for you', 'Why are you feeling bad', 'What is wrong with you',
 'You totally deserve this prize', 'Let us go play football',
 'Are you down for football this afternoon', 'Work hard play harder',
 'It is suprising how people can be dumb sometimes',
 'I am very disappointed','It is the best day in my life',
 'I think I will end up alone','My life is so boring','Good job',
 'Great so awesome'])

Y_train = np.asarray([5,0,0,5, 4, 4, 4, 6, 6, 4, 1, 1, 5, 6, 6, 3, 6, 3, 4, 4])
Y_hot_array = pd.get_dummies(pd.Series(Y_train)).values

le_classes_demo = ['fun', "love", "miss", "enthusiasm", "sad", "worry"]

pred, W, b = model(X_train, Y_hot_array, word_to_vec_map, le_classes_demo)

Epoch: 0 --- cost = 2.173863787748305
Epoch: 100 --- cost = 0.30368059361537286
Epoch: 200 --- cost = 0.13781914469884504
Epoch: 300 --- cost = 0.08622735005880636


In [15]:
train_data_Y_hot_array = pd.get_dummies(pd.Series(train_data_le_y)).values
test_data_Y_hot_array = pd.get_dummies(pd.Series(test_data_le_y)).values

In [16]:
pred_data, W_parm, b_param = model(train_data["content"].values, train_data_Y_hot_array, word_to_vec_map, le_classes)

Epoch: 0 --- cost = 1.9722075841367954
Epoch: 100 --- cost = 0.13943084030253142
Epoch: 200 --- cost = 0.10314076067129022
Epoch: 300 --- cost = 0.09405915179524556


In [17]:
pred_test_data_values = predict(test_data["content"].values, test_data_Y_hot_array, W_parm, b_param, word_to_vec_map, le_classes)
actual_test_data_values = test_data["sentiment"].values

In [18]:
X_my_sentences = np.array(["i adore you", "i love you", "funny lol", "lets play with a ball", "food is ready", "not feeling happy"])

In [19]:
pred_test_data_values1 = predict1(X_my_sentences, W_parm, b_param, word_to_vec_map, le_classes)


In [20]:
pred_test_data_values1

['happiness', 'love', 'love', 'love', 'anger', 'sadness']

In [21]:
actual_test_data_values

array(['hate', 'sadness', 'love', 'sadness', 'love', 'love', 'happiness',
       'anger', 'hate', 'happiness', 'anger', 'anger', 'happiness',
       'love', 'happiness', 'love', 'happiness', 'anger', 'hate', 'love',
       'hate', 'happiness', 'hate', 'hate', 'happiness', 'hate',
       'happiness', 'happiness', 'happiness', 'sadness', 'love',
       'happiness', 'sadness', 'hate', 'hate', 'love', 'love', 'love',
       'hate', 'love', 'anger', 'anger', 'hate', 'sadness', 'happiness',
       'anger', 'anger', 'hate', 'love', 'hate', 'sadness', 'hate',
       'love', 'sadness', 'love', 'love', 'hate', 'anger', 'sadness',
       'love', 'love', 'hate', 'love', 'sadness', 'anger', 'happiness',
       'sadness', 'love', 'happiness', 'sadness', 'hate', 'happiness',
       'sadness', 'happiness', 'anger', 'happiness', 'anger', 'hate',
       'sadness', 'love', 'hate', 'sadness', 'sadness', 'hate',
       'happiness', 'hate', 'hate', 'sadness', 'hate', 'love', 'anger',
       'happiness', 'sa

In [22]:
pred_test_data_values

['anger',
 'love',
 'love',
 'anger',
 'anger',
 'love',
 'hate',
 'love',
 'anger',
 'love',
 'sadness',
 'happiness',
 'hate',
 'sadness',
 'sadness',
 'happiness',
 'happiness',
 'sadness',
 'hate',
 'happiness',
 'sadness',
 'love',
 'hate',
 'hate',
 'love',
 'hate',
 'happiness',
 'sadness',
 'sadness',
 'hate',
 'happiness',
 'hate',
 'happiness',
 'happiness',
 'anger',
 'sadness',
 'love',
 'love',
 'sadness',
 'happiness',
 'hate',
 'sadness',
 'hate',
 'sadness',
 'love',
 'anger',
 'hate',
 'anger',
 'happiness',
 'happiness',
 'love',
 'hate',
 'happiness',
 'love',
 'love',
 'happiness',
 'sadness',
 'hate',
 'love',
 'happiness',
 'love',
 'love',
 'love',
 'hate',
 'hate',
 'happiness',
 'sadness',
 'hate',
 'sadness',
 'hate',
 'sadness',
 'anger',
 'love',
 'happiness',
 'sadness',
 'love',
 'sadness',
 'hate',
 'hate',
 'love',
 'anger',
 'anger',
 'anger',
 'hate',
 'love',
 'anger',
 'hate',
 'love',
 'anger',
 'happiness',
 'sadness',
 'love',
 'love',
 'happiness